# Evaluation Setup

The following setup is based on our methodology described in the paper
**_“Towards Standardization of the Earth Observation Data Product Supply Chain – Are OCI Artifacts the Key to Ubiquitous and Scalable EO Data Handling?”_**

→ Read the [full paper](oci-supply-chain.pdf) (PDF)

## Loading Libraries
Make sure you have installed the requirements, e.g. with `pip install pystac rasterio`

In [ ]:
import time
import pystac
import rasterio
from pathlib import Path